# Цель лабораторной работы

Изучить сложные способы подготовки выборки и подбора гиперпараметров на примере метода ближайших соседей.

# Задание

Требуется выполнить следующие действия:

1. Выбрать набор данных (датасет) для решения задачи классификации или регресии.
2. В случае необходимости проведите удаление или заполнение пропусков и кодирование категориальных признаков.
3. С использованием метода train_test_split разделите выборку на обучающую и тестовую.
4. Обучите модель ближайших соседей для произвольно заданного гиперпараметра
5. Оцените качество модели с помощью трех подходящих для задачи метрик.
6. Постройте модель и оцените качество модели с использованием кросс-валидации. Проведите эксперименты с тремя различными стратегиями кросс-валидации.
7. Произведите подбор гиперпараметра с использованием GridSearchCV и кросс-валидации.
8. Сравните качество полученной модели с качеством модели, полученной в пункте 4.

# Ход выполнения работы

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import * 
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error, mean_squared_error, median_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
data = pd.read_csv('heart_failure_clinical_records_dataset.csv')

In [3]:
dataCopy = data.copy()

In [4]:
data.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [5]:
#Кодирование категориальных признаков
data["anaemia"] = data["anaemia"].astype('category')
data["diabetes"] = data["diabetes"].astype('category')

#Назначить закодированную переменную новосу столбцу с помощью метода доступа
data["anaemia_cat"] = data["anaemia"].cat.codes
data["diabetes_cat"] = data["diabetes"].cat.codes

In [6]:
data = data.drop(['anaemia', 'diabetes'], axis=1)

In [7]:
data.head()

,age,creatinine_phosphokinase,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT,anaemia_cat,diabetes_cat
0,75.0,582,20,1,265000.00,1.9,130,1,0,4,1,0,0
1,55.0,7861,38,0,263358.03,1.1,136,1,0,6,1,0,0
2,65.0,146,20,0,162000.00,1.3,129,1,1,7,1,0,0
3,50.0,111,20,0,210000.00,1.9,137,1,0,7,1,1,0
4,65.0,160,20,0,327000.00,2.7,116,0,0,8,1,1,1


In [8]:
data.isna().sum()

age                         0
creatinine_phosphokinase    0
ejection_fraction           0
high_blood_pressure         0
platelets                   0
serum_creatinine            0
serum_sodium                0
sex                         0
smoking                     0
time                        0
DEATH_EVENT                 0
anaemia_cat                 0
diabetes_cat                0
dtype: int64

In [9]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
age,299.0,60.833893,11.894809,40.0,51.0,60.0,70.0,95.0
creatinine_phosphokinase,299.0,581.839465,970.287881,23.0,116.5,250.0,582.0,7861.0
ejection_fraction,299.0,38.083612,11.834841,14.0,30.0,38.0,45.0,80.0
high_blood_pressure,299.0,0.351171,0.478136,0.0,0.0,0.0,1.0,1.0
platelets,299.0,263358.029264,97804.236869,25100.0,212500.0,262000.0,303500.0,850000.0
serum_creatinine,299.0,1.393880,1.034510,0.5,0.9,1.1,1.4,9.4
serum_sodium,299.0,136.625418,4.412477,113.0,134.0,137.0,140.0,148.0
sex,299.0,0.648829,0.478136,0.0,0.0,1.0,1.0,1.0
smoking,299.0,0.321070,0.467670,0.0,0.0,0.0,1.0,1.0
time,299.0,130.260870,77.614208,4.0,73.0,115.0,203.0,285.0


Разделение данных

In [10]:
y=data['sex'] #Наименования признаков
x=data.drop('sex', axis=True) # Значения признаков

In [11]:
# random_state позволяет задавать базовое значение для генератора случайных чисел, чтобы сделать выборку неслучайной 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=45)

In [12]:
# Размер обучающей выборки
x_train.shape, y_train.shape

((224, 12), (224,))

In [13]:
# Размер тестовой выборки
x_test.shape, y_test.shape

((75, 12), (75,))

In [14]:
np.unique(y_train)

array([0, 1], dtype=int64)

In [15]:
np.unique(y_test)

array([0, 1], dtype=int64)

Напишем функцию, которая считает метрики построенной модели:

In [16]:
#Масштабирование данных
scaler=MinMaxScaler().fit(x_train)
x_train=pd.DataFrame(scaler.transform(x_train), columns=x_train.columns)
x_test=pd.DataFrame(scaler.transform(x_test), columns=x_test.columns)
x_train.describe()

,age,creatinine_phosphokinase,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,smoking,time,DEATH_EVENT,anaemia_cat,diabetes_cat
count,224.000000,224.000000,224.000000,224.000000,224.000000,224.000000,224.000000,224.000000,224.000000,224.000000,224.000000,224.000000
mean,0.379654,0.071233,0.366477,0.348214,0.285454,0.103692,0.578208,0.325893,0.445947,0.330357,0.464286,0.428571
std,0.226637,0.122863,0.181569,0.477471,0.127055,0.129110,0.152551,0.469757,0.276568,0.471395,0.499840,0.495980
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.181818,0.011981,0.242424,0.000000,0.216572,0.044944,0.481481,0.000000,0.237410,0.000000,0.000000,0.000000
50%,0.363636,0.029496,0.363636,0.000000,0.276882,0.067416,0.592593,0.000000,0.384892,0.000000,0.000000,0.000000
75%,0.545455,0.072796,0.469697,1.000000,0.337192,0.092697,0.703704,1.000000,0.713129,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Метрики построенной модели

In [17]:
def test_model(model):
    print("mean_absolute_error: ", mean_absolute_error(y_test, model.predict(x_test)))
    print("mean_squared_error: ", mean_squared_error(y_test, model.predict(x_test)))
    print("median_absolute_error: ", median_absolute_error(y_test, model.predict(x_test)))
    print("r2_score: ", r2_score(y_test, model.predict(x_test)))

Метод ближайших соседей с гиперпараметром K=5

In [18]:
reg_5=KNeighborsRegressor(n_neighbors=5)
reg_5.fit(x_train, y_train)

KNeighborsRegressor()

In [19]:
test_model(reg_5)

mean_absolute_error:  0.37599999999999995
mean_squared_error:  0.20746666666666666
median_absolute_error:  0.4
r2_score:  0.04656862745098034


mean_absolute_error: 0.376, чем ближе к нулю, тем выше качество регрессии

mean_squared_error: 0.2, чем ближе к нулю, тем лучше модель

median_absolute_error: 0.4

r2_score: 0.05, чем ближе к 0, тем слабее зависимость, то есть разброс предсказаний модели y относительно разброса самой целевой переменной y достаточно велик - то есть модель плохо описывает данные